In [1]:
using PCT

[ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]


In [2]:
f, ctx = @pct begin
    @space V begin
        type=(I,)->C
    end

    @space H begin
        type=(I,I)->C
    end
    (A::H) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H)), :A)]), Var{UndeterminedPCTType}(UndeterminedPCTType(), :_)), TypeContext(Dict{Symbol, Vector{<:AbstractPCTType}}(:_ => AbstractPCTType[UndeterminedPCTType()], :H => AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))], :V => AbstractPCTType[MapType(VecType(AbstractPCTType[I()]), C(), Dict(:name => :V))])))

In [4]:
g = fc(@pct f ctx (x::R) -> x)

(x) -> 
    x

In [5]:
cg = decompose(g)

x: 


In [6]:
as_map(cg)

(x) -> 
    x

In [7]:
eval_all(pp(cg))

(x, _k_0) -> 
    _k_0

In [62]:
g = fc(@pct f ctx (x::R) -> 1 +  x*x)

(x) -> 
    ((x⋅x)+1)

In [63]:
cg = decompose(g)

x: +1 ▷ *x


In [64]:
as_map(cg)

(x) -> 
    ((x⋅x)+1)

In [65]:
pp(cg)

----------------
+1
(_z_0, _k_0) -> 
    _k_0
----------------
*x
(_z_0, _k_0) -> 
    (x⋅_k_0)
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)
----------------
{_z_0 ⇥ x: *_z_0}
(_z_1, _k_0) -> 
    ∑((_z_0), (_z_0⋅_k_0(_z_0)))
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)


(x, _k_0) -> 
    ((x⋅_k_0)+∑((_z_0), (δ(x, _z_0, _k_0)⋅_z_0)))

In [66]:
eval_all(pp(cg))

----------------
+1
(_z_0, _k_0) -> 
    _k_0
----------------
*x
(_z_0, _k_0) -> 
    (x⋅_k_0)
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)
----------------
{_z_0 ⇥ x: *_z_0}
(_z_1, _k_0) -> 
    ∑((_z_0), (_z_0⋅_k_0(_z_0)))
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)


(x, _k_0) -> 
    ((x⋅_k_0)+∑((_z_0), (δ(x, _z_0, _k_0)⋅_z_0)))

In [67]:
simplify(eval_all(pp(cg))) |> first

----------------
+1
(_z_0, _k_0) -> 
    _k_0
----------------
*x
(_z_0, _k_0) -> 
    (x⋅_k_0)
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)
----------------
{_z_0 ⇥ x: *_z_0}
(_z_1, _k_0) -> 
    ∑((_z_0), (_z_0⋅_k_0(_z_0)))
----------------
*_z_0
(_z_1, _k_0) -> 
    (_z_0⋅_k_0)
  0.000001 seconds
  0.000100 seconds (345 allocations: 13.000 KiB)
1 swallow
∑((_z_0), (δ(x, _z_0, _k_0)⋅_z_0))
∑((_z_0), δ(_z_0, x, (_z_0⋅_k_0)))
-->

  0.000005 seconds (4 allocations: 192 bytes)
2 contract_delta
∑((_z_0), δ(_z_0, x, (_z_0⋅_k_0)))
(x⋅_k_0)
-->

  0.000000 seconds
  0.000000 seconds


(x, _k_0) -> 
    (2.0⋅x⋅_k_0)

In [20]:
g = fc(@pct f ctx (x::V) -> sum(i, x(i)))

(x) -> 
    ∑((i), x(i))

In [21]:
cg = decompose(g)

x: ∑i ▷ {i ⇥ x: ℳ x ▷ ->i}


In [22]:
eval_all(as_map(cg))

(x) -> 
    (i) -> 
        i

In [26]:
pp(cg)

(x, _k_0) -> 
    (_i_0) -> 
        ∑((i), δ(i, _i_0, _k_0))

In [27]:
simplify(pp(cg)) |> first

  0.000020 seconds (2 allocations: 112 bytes)
1 contract_delta
∑((i), δ(i, _i_0, _k_0))
_k_0
yesdirected!

  0.000000 seconds


(x, _k_0) -> 
    (_i_0) -> 
        _k_0

In [28]:
g = fc(@pct f ctx (x::V) -> sum(i, x(i)' * x(i)))

(x) -> 
    ∑((i), (x(i)'⋅x(i)))

In [29]:
cg = decompose(g)

x: ∑i ▷ {i ⇥ x: *x(i) ▷ † ▷ ℳ x ▷ ->i}


In [34]:
eval_all(as_map(cg))

(x) -> 
    ∑((i), (x(i)'⋅x(i)))

In [82]:
pp(cg)

(_z_0, _k_0) -> 
    (_i_1) -> 
        ∑((i), (∑((_i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))+δ(i, _i_1, (_k_0'⋅x(i)))))

In [83]:
simplify(eval_all(pp(cg))) |> first

  0.000398 seconds (2.81 k allocations: 103.969 KiB)
1 sum_dist
∑((i), (∑((_i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))+δ(i, _i_1, (_k_0'⋅x(i)))))
(∑((i), δ(i, _i_1, (_k_0'⋅x(i))))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
-->

  0.000256 seconds (2.57 k allocations: 80.125 KiB)
2 contract_delta
(∑((i), δ(i, _i_1, (_k_0'⋅x(i))))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
((_k_0'⋅x(_i_1))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
-->

  0.000185 seconds (1.71 k allocations: 53.656 KiB)
3 contract_delta
((_k_0'⋅x(_i_1))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
((_k_0'⋅x(_i_1))+∑((_i_3), (_i_3'⋅δ(x(_i_1)', _i_3, _k_0))))
-->

  0.000120 seconds (1.15 k allocations: 36.547 KiB)
4 swallow
((_k_0'⋅x(_i_1))+∑((_i_3), (_i_3'⋅δ(x(_i_1)', _i_3, _k_0))))
((_k_0'⋅x(_i_1))+∑((_i_3), δ(_i_3, x(_i_1)', (_k_0⋅_i_3'))))
-->

  0.000012 seconds (14 allocations: 512 bytes)
5 contract_delta
((_k_0'⋅x(_i_1))+∑((_i_3), δ(_i_3, x(_i_1)', (_k_0⋅_i_3'

(_z_0, _k_0) -> 
    (_i_1) -> 
        (x(_i_1)⋅(_k_0+_k_0'))

In [84]:
g = fc(@pct f ctx (x::C) -> x' * x)

(x) -> 
    (x⋅x')

In [85]:
cg = decompose(g)

x: *x'


In [86]:
pp(cg)

(_z_0, _k_0) -> 
    ((x⋅_k_0)+∑((_i_0), (_i_0⋅δ(_i_0, x, _k_0'))))

In [87]:
simplify(pp(cg); settings=Dict(:clench_sum=>true)) |> first

  0.000122 seconds (824 allocations: 26.594 KiB)
1 swallow
((x⋅_k_0)+∑((_i_0), (_i_0⋅δ(_i_0, x, _k_0'))))
((x⋅_k_0)+∑((_i_0), δ(x, _i_0, (_i_0⋅_k_0'))))
-->

  0.000012 seconds (10 allocations: 384 bytes)
2 contract_delta
((x⋅_k_0)+∑((_i_0), δ(x, _i_0, (_i_0⋅_k_0'))))
((x⋅_k_0)+(x⋅_k_0'))
-->

  0.000003 seconds (6 allocations: 256 bytes)
3 gcd
((x⋅_k_0)+(x⋅_k_0'))
(x⋅(_k_0+_k_0'))
-->

  0.000000 seconds


(_z_0, _k_0) -> 
    (x⋅(_k_0+_k_0'))

In [80]:
sort([], by=e->-e[2])

Any[]